In [1]:
%load_ext autoreload
%autoreload 2
import warnings
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = 'last_expr'

In [3]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
mlflow_uri = config['mlflow']['mlflow_uri']
mlflow_artifact_uri = config['mlflow']['target_mlflow_artifact_uri']

In [4]:
import flash
from flash.core.data.utils import download_data

download_data(config['text-example']['download_url'], 'data/')

In [5]:
from flash.text import TextClassificationData, TextClassifier

input_fields,target_fields = 'review', 'sentiment'
batch_size=512
datamodule = TextClassificationData.from_csv(
    train_file=config['text-example']['train_file'],
    val_file=config['text-example']['val_file'],
    test_file=config['text-example']['test_file'],
    input_fields=input_fields,
    target_fields=target_fields,
    batch_size=batch_size
)

Using custom data configuration default
100%|██████████| 23/23 [00:29<00:00,  1.30s/ba]
Using custom data configuration default
100%|██████████| 3/3 [00:03<00:00,  1.26s/ba]
Using custom data configuration default
100%|██████████| 3/3 [00:03<00:00,  1.09s/ba]


In [6]:
model = TextClassifier(num_classes=datamodule.num_classes, backbone='prajjwal1/bert-mini')

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

**Backbone** Can be seen on huggingface: [huggingface-link](https://huggingface.co/prajjwal1/bert-mini)

In [10]:
import os
from pytorch_lightning.loggers import MLFlowLogger

username = os.getenv("uid")
mlf_logger = MLFlowLogger(experiment_name=f'lightning-flash-{username}-bert', tracking_uri=mlflow_uri)

In [11]:
 trainer = flash.Trainer(logger=mlf_logger, max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [12]:
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

Experiment with name lightning-flash-chengb-bert not found. Creating it.

  | Name    | Type                          | Params
----------------------------------------------------------
0 | metrics | ModuleDict                    | 0     
1 | model   | BertForSequenceClassification | 11.2 M
----------------------------------------------------------
514       Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.684    Total estimated model params size (MB)
Epoch 0:  92%|█████████▏| 44/48 [12:48<01:09, 17.46s/it, loss=0.648, v_num=da32, val_accuracy=0.504, val_cross_entropy=0.723, train_accuracy_step=0.666, train_cross_entropy_step=0.628]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 48/48 [14:10<00:00, 17.71s/it, loss=0.648, v_num=da32, val_accuracy=0.675, val_cross_entropy=0.612, train_accuracy_step=0.643, train_cross_entropy_step=0.645, train_accuracy_epoch=0.578, train_cross_entropy_epoch=0.718]


In [13]:
trainer.test()

Testing: 100%|██████████| 5/5 [01:08<00:00, 13.78s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6736000180244446, 'test_cross_entropy': 0.6115491986274719}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.6736000180244446,
  'test_cross_entropy': 0.6115491986274719}]

In [18]:
# If one is not training on linux blade and training on windows laptop
import shutil
import platform

if platform.system() == 'Windows':
    file_names = os.listdir()
    exp_num = '138'
    current_artifact_base_fpath = os.getcwd()
    source_dir = f'{current_artifact_base_fpath}'
    target_dir = mlflow_artifact_uri
    for file_name in file_names:
        if file_name  == exp_num:
            for subfolder in os.listdir(file_name):
                for artifacts in os.listdir(os.path.join(file_name,subfolder)):
                    if artifacts == 'checkpoints':
                        orig_artifacts_loc = os.path.join(os.path.join(os.path.join(source_dir, file_name), subfolder), artifacts)
                        # print(orig_artifacts_loc)
                        new_artifacts_loc = os.path.join(os.path.join(os.path.join(source_dir, file_name), subfolder), 'artifacts')
                        # print(new_artifacts_loc)
                        shutil.move(orig_artifacts_loc, new_artifacts_loc)
            shutil.move(os.path.join(source_dir, file_name), target_dir)

#### Take a look at mlflow for best run 

In [19]:
run_id = '1b5c9eb454c44109b8977a37d21eda32'
model_path = f'{mlflow_artifact_uri}\\{exp_num}\\{run_id}\\artifacts'
for pt in os.listdir(model_path):
    if pt.endswith('.ckpt'):
        ckpt_path = f'{model_path}\\{pt}'

In [20]:
model = TextClassifier.load_from_checkpoint(ckpt_path)

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [21]:
# pred_datamodule = TextClassificationData.from_csv(
#     predict_file=config['text-example']['predict_file'],
#     input_fields=input_fields,
# )
# predictions = flash.Trainer().predict(model, datamodule=pred_datamodule)
# print(predictions)

predictions = model.predict([
    "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
    "I come from Bulgaria where it 's almost impossible to have a tornado."
    "Very, very afraid"
    "This guy has done a great job with this movie!",
])
print(predictions)

['negative', 'positive']
